In [73]:
from urllib.request import urlopen
import urllib.parse
from bs4 import BeautifulSoup

from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

from IPython.core.display import display, HTML

import pymysql
import threading
from time import sleep

In [74]:
stopwords = ["중인", "만큼", "마찬가지", "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자","아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가", "으로", "로", "에게", "뿐이다", "의거하여", "근거하여", "입각하여", "기준으로", "예하면", "예를", "들면", "들자면", "저", "소인", "저희", "지말고", "하지마", "하지마라", "다른", "물론", "또한", "그리고", "비길수", "없다", "해서는", "안된다", "뿐만", "아니라", "만이", "아니다", "만은", "아니다", "막론하고", "관계없이", "그치지", "않다", "그러나", "그런데", "하지만", "든간에", "논하지", "않다", "따지지", "않다", "설사", "비록", "더라도", "아니면", "만", "못하다", "하는", "편이", "낫다", "불문하고", "향하여", "향해서", "향하다", "쪽으로", "틈타", "이용하여", "타다", "오르다", "제외하고", "이", "외에", "이", "밖에", "하여야", "비로소", "한다면", "몰라도", "외에도", "이곳", "여기", "부터", "기점으로", "따라서", "할", "생각이다", "하려고하다", "이리하여", "그리하여", "그렇게", "함으로써", "하지만", "일때", "할때", "앞에서", "중에서", "보는데서", "으로써", "로써", "까지", "해야한다", "일것이다", "반드시", "할줄알다", "할수있다", "할수있어", "임에", "틀림없다", "한다면", "등", "등등", "제", "겨우", "단지", "다만", "할뿐", "딩동", "댕그", "대해서", "대하여", "대하면", "훨씬", "얼마나", "얼마만큼", "얼마큼", "남짓", "여", "얼마간", "약간", "다소", "좀", "조금", "다수", "몇", "얼마", "지만", "하물며", "또한", "그러나", "그렇지만", "하지만", "이외에도", "대해", "말하자면", "뿐이다", "다음에", "반대로", "반대로", "말하자면", "이와", "반대로", "바꾸어서", "말하면", "바꾸어서", "한다면", "만약", "그렇지않으면", "까악", "툭", "딱", "삐걱거리다", "보드득", "비걱거리다", "꽈당", "응당", "해야한다", "에", "가서", "각", "각각", "여러분", "각종", "각자", "제각기", "하도록하다", "와", "과", "그러므로", "그래서", "고로", "한", "까닭에", "하기", "때문에", "거니와", "이지만", "대하여", "관하여", "관한", "과연", "실로", "아니나다를가", "생각한대로", "진짜로", "한적이있다", "하곤하였다", "하", "하하", "허허", "아하", "거바", "와", "오", "왜", "어째서", "무엇때문에", "어찌", "하겠는가", "무슨", "어디", "어느곳", "더군다나", "하물며", "더욱이는", "어느때", "언제", "야", "이봐", "어이", "여보시오", "흐흐", "흥", "휴", "헉헉", "헐떡헐떡", "영차", "여차", "어기여차", "끙끙", "아야", "앗", "아야", "콸콸", "졸졸", "좍좍", "뚝뚝", "주룩주룩", "솨", "우르르", "그래도", "또", "그리고", "바꾸어말하면", "바꾸어말하자면", "혹은", "혹시", "답다", "및", "그에", "따르는", "때가", "되어", "즉", "지든지", "설령", "가령", "하더라도", "할지라도", "일지라도", "지든지", "몇", "거의", "하마터면", "인젠", "이젠", "된바에야", "된이상", "만큼 어찌됏든", "그위에", "게다가", "점에서", "보아", "비추어", "보아", "고려하면", "하게될것이다", "일것이다", "비교적", "좀", "보다더", "비하면", "시키다", "하게하다", "할만하다", "의해서", "연이서", "이어서", "잇따라", "뒤따라", "뒤이어", "결국", "의지하여", "기대여", "통하여", "자마자", "더욱더", "불구하고", "얼마든지", "마음대로", "주저하지", "않고", "곧", "즉시", "바로", "당장", "하자마자", "밖에", "안된다", "하면된다", "그래", "그렇지", "요컨대", "다시", "말하자면", "바꿔", "말하면", "즉", "구체적으로", "말하자면", "시작하여", "시초에", "이상", "허", "헉", "허걱", "바와같이", "해도좋다", "해도된다", "게다가", "더구나", "하물며", "와르르", "팍", "퍽", "펄렁", "동안", "이래", "하고있었다", "이었다", "에서", "로부터", "까지", "예하면", "했어요", "해요", "함께", "같이", "더불어", "마저", "마저도", "양자", "모두", "습니다", "가까스로", "하려고하다", "즈음하여", "다른", "다른", "방면으로", "해봐요", "습니까", "했어요", "말할것도", "없고", "무릎쓰고", "개의치않고", "하는것만", "못하다", "하는것이", "낫다", "매", "매번", "들", "모", "어느것", "어느", "로써", "갖고말하자면", "어디", "어느쪽", "어느것", "어느해", "어느", "년도", "라", "해도", "언젠가", "어떤것", "어느것", "저기", "저쪽", "저것", "그때", "그럼", "그러면", "요만한걸", "그래", "그때", "저것만큼", "그저", "이르기까지", "할", "줄", "안다", "할", "힘이", "있다", "너", "너희", "당신", "어찌", "설마", "차라리", "할지언정", "할지라도", "할망정", "할지언정", "구토하다", "게우다", "토하다", "메쓰겁다", "옆사람", "퉤", "쳇", "의거하여", "근거하여", "의해", "따라", "힘입어", "그", "다음", "버금", "두번째로", "기타", "첫번째로", "나머지는", "그중에서", "견지에서", "형식으로", "쓰여", "입장에서", "위해서", "단지", "의해되다", "하도록시키다", "뿐만아니라", "반대로", "전후", "전자", "앞의것", "잠시", "잠깐", "하면서", "그렇지만", "다음에", "그러한즉", "그런즉", "남들", "아무거나", "어찌하든지", "같다", "비슷하다", "예컨대", "이럴정도로", "어떻게", "만약", "만일", "위에서", "서술한바와같이", "인", "듯하다", "하지", "않는다면", "만약에", "무엇", "무슨", "어느", "어떤", "아래윗", "조차", "한데", "그럼에도", "불구하고", "여전히", "심지어", "까지도", "조차도", "하지", "않도록", "않기", "위하여", "때", "시각", "무렵", "시간", "동안", "어때", "어떠한", "하여금", "네", "예", "우선", "누구", "누가", "알겠는가", "아무도", "줄은모른다", "줄은", "몰랏다", "하는", "김에", "겸사겸사", "하는바", "그런", "까닭에", "한", "이유는", "그러니", "그러니까", "때문에", "그", "너희", "그들", "너희들", "타인", "것", "것들", "너", "위하여", "공동으로", "동시에", "하기", "위하여", "어찌하여", "무엇때문에", "붕붕", "윙윙", "나", "우리", "엉엉", "휘익", "윙윙", "오호", "아하", "어쨋든", "만", "못하다", "하기보다는", "차라리", "하는", "편이", "낫다", "흐흐", "놀라다", "상대적으로", "말하자면", "마치", "아니라면", "쉿", "그렇지", "않으면", "그렇지", "않다면", "안", "그러면", "아니었다면", "하든지", "아니면", "이라면", "좋아", "알았어", "하는것도", "그만이다", "어쩔수", "없다", "하나", "일", "일반적으로", "일단", "한켠으로는", "오자마자", "이렇게되면", "이와같다면", "전부", "한마디", "한항목", "근거로", "하기에", "아울러", "하지", "않도록", "않기", "위해서", "이르기까지", "이", "되다", "로", "인하여", "까닭으로", "이유만으로", "이로", "인하여", "그래서", "이", "때문에", "그러므로", "그런", "까닭에", "알", "수", "있다", "결론을", "낼", "수", "있다", "으로", "인하여", "있다", "어떤것", "관계가", "있다", "관련이", "있다", "연관되다", "어떤것들", "에", "대해", "이리하여", "그리하여", "여부", "하기보다는", "하느니", "하면", "할수록", "운운", "이러이러하다", "하구나", "하도다", "다시말하면", "다음으로", "에", "있다", "에", "달려", "있다", "우리", "우리들", "오히려", "하기는한데", "어떻게", "어떻해", "어찌됏어", "어때", "어째서", "본대로", "자", "이", "이쪽", "여기", "이것", "이번", "이렇게말하자면", "이런", "이러한", "이와", "같은", "요만큼", "요만한", "것", "얼마", "안", "되는", "것", "이만큼", "이", "정도의", "이렇게", "많은", "것", "이와", "같다", "이때", "이렇구나", "것과", "같이", "끼익", "삐걱", "따위", "와", "같은", "사람들", "부류의", "사람들", "왜냐하면", "중의하나", "오직", "오로지", "에", "한하다", "하기만", "하면", "도착하다", "까지", "미치다", "도달하다", "정도에", "이르다", "할", "지경이다", "결과에", "이르다", "관해서는", "여러분", "하고", "있다", "한", "후", "혼자", "자기", "자기집", "자신", "우에", "종합한것과같이", "총적으로", "보면", "총적으로", "말하면", "총적으로", "대로", "하다", "으로서", "참", "그만이다", "할", "따름이다", "쿵", "탕탕", "쾅쾅", "둥둥", "봐", "봐라", "아이야", "아니", "와아", "응", "아이", "참나", "년", "월", "일", "령", "영", "일", "이", "삼", "사", "오", "육", "륙", "칠", "팔", "구", "이천육", "이천칠", "이천팔", "이천구", "하나", "둘", "셋", "넷", "다섯", "여섯", "일곱", "여덟", "아홉", "령", "영"]

In [75]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.twitter = Okt()
  
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        '''
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        '''
        for (i,s) in enumerate(sentences):
            if '@' in s:
                del sentences[i]
        sentences.pop()
        return sentences

    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                        if noun not in stopwords and len(noun) > 1]))

        return nouns

In [76]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []

    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence

In [77]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1

        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

In [78]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
    
        self.sentences = self.sent_tokenize.text2sentences(text)

        self.nouns = self.sent_tokenize.get_nouns(self.sentences)

        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)

        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)


    def summarize(self, sent_num=1):
        summary = []
        num = len(self.sentences)
        if(num>=50):
            sent_num = 5
        if(num>=3):
            sent_num = 3
              
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)

        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])

        return summary

In [79]:
def get_art_body(URL):
  a = Article(URL,language='ko')
  a.download()
  a.parse()
  return (a.title,a.text)

def get_articles_list():  
  with urllib.request.urlopen("https://www.hankyung.com/society?hkonly=true") as response:
    html = response.read()
    soup = BeautifulSoup(html,'html.parser')
    art_link = soup.select('div.article > span > a')

  articles = []

  for (i,a) in enumerate(art_link):
    #articles[i][0]: 링크, articles[i][1]: 제목, articles[i][2]: 본문
    if i == 15:
      break
    l = a.get('href')
    (t,c) = get_art_body(l)
    r = TextRank(c).summarize()
    r = "-".join(r)
    articles.append([t,r,l])  #[[제목,텍스트랭크,링크],...]]
    

  return articles

In [80]:
def insert_db():
    conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='171217', db='test', charset='utf8')
    atcList = get_articles_list()
    try:
      for a in atcList:
        with conn.cursor() as cursor:
          sql = 'INSERT INTO news (title,content,url) VALUES (%s, %s, %s)'
          cursor.execute(sql, (a[0],a[1],a[2]))
        conn.commit()
        print(cursor.lastrowid)

    finally:
        conn.close()
    print("DB insert 성공")


In [81]:
def myThread(name,nsec):
    insert_db()
    sleep(nsec)
    
    if __name__ == '__main__':
        t = threading.Thread(target=myThread, args=("Thread-1",3))
        t.start()
        t.join()
        print("----e----")
        
myThread("hi",3000)


['충북· 충남서 전출 신청 교사 5년 간 6000명 넘어 학령인구도 급감 예상 교사들의 지방 기피 현상이 심각한 수준인 것으로 나타났다.', '지방을 중심으로 급격하게 학령인구가 줄어드는 상황에서 자칫 지방 교육 공동화( 空洞化) 현상으로 이어질 것이라는 우려가 나온다.', '국회 교육위원회 소속 박 용진 더불어 민주당 의원이 11일 17개 시· 도교육청에서 받은 ‘ 최근 5년 간 교원의 타지역 전출 신청 현황’ 자료에 따르면 충북 (3162 명) 과 충남 (3048 명 )에서 6000명이 넘는 교원이 다른 지역으로 전출을 신청한 것으로 나타났다.', '경북과 경남도 각각 2861명, 2835명의 교원이 타지역 전출을 희망했다.', '전출을 신청한 교원은 수도권 및 인근 대도시로 전입하려는 경향이 뚜렷했다.', '충북 지역에서 전출을 신청한 교원 중에서는 경기로 전입을 신청한 인원이 806명으로 가장 많았다.', '4명 중 1명이 경기 지역 전입을 희망한 셈이다.', '세종 (689 명) 과 서울 (406 명), 대전 (363 명) 이 뒤를 이었다.', '강원 지역에서 전출을 신청한 교원 중 67.7% 는 서울과 경기, 인천 등 수도권 지역으로 전입을 희망했다.', '대도시 및 수도권으로 전출 신청 교원이 많은 충북과 강원 지역 등은 학령인구 역시 급격하게 줄어들 것으로 전망된다.', '충북 지역의 학령인구는 2020년 16만 9560명에서 2029년 14만 6190명으로 13.8% 줄어들 것으로 예상된다.']
['부산시 국감서 답변 민정 수석실 감찰 무마 의혹 해명 유 재수 부산시 부시장( 사진) 은 11일 “ 조국 장관과는 알지 못하는 사이” 라며 “ 공직에 사표를 낸 것은 구체적으로 이야기할 수는 없지만 경미한 품위 위반 때문에 제출했고, 잘못에 대한 대가를 이미 치렀다고', '생각한다” 고 말했다.', '유 부시장은 이날 부산 시청에서 열린 부산시 국정감사에서 “ 민정 수석실의 감찰에서 비리는 없었다 ”며 이같이 답했다.', '유 부시장은 조국 법무부

['" 曺 장관 수사 위헌적 쿠데타 사퇴 집회에 한국당 손길" 발언 시민단체 등 서 지난달 고발 유시민 사람 사는 세상 노무현재단 이사장( 사진) 이 검찰의 조국 법무부 장관 일가 수사 등과 관련해 허위사실을 유포했다는 혐의로 고발당한 사건들에 대해 검찰이 수사에 들어갔다.11', '일 법조계에 따르면 서울 서부지방 검찰청은 유 이사장이 지난달 검찰의 조 장관 수사를 두고 “ 총 칼은 안 들었지만 위헌적 쿠데타나 마찬가지 ”라고 했다가 한 시민단체로부터 고발당한 사건을 지난 1일 형사 3부( 부장검사 이재 승 )에 배당했다.', '검찰은 전날 한 차례 고발인 조사를 마친 것으로 알려 졌다.', '유 이사장은 지난달 28일 경남 창원시에서 열린 한 강연 자리에서 검찰의 조 장관 수사를 ‘ 윤석 열의 난’ 이라 표현하며 “ 검찰 조직에 남아 있는 ‘ 우리가 나라를 구해야 한다’ 는 식의 전두환 신군부와 비슷한 정서가 현재 상황을 만들었다” 고 발언했다.', '앞서 그는 개인 유 튜브 방송에서 조 장관 부인인 정 경심 동양 대 교수의 ‘PC 무단 반출 의혹 ’에 대해 “ 증거 인멸이 아니라 증거를 지키기 위한 것” 이라며 “ 검찰이 압수 수색해서 장난칠 경우를 대비하기 위해 동양 대 컴퓨터, 집 컴퓨터를 복제하려고 반출한 것” 이라고 주장했다.', '이에 시민단체인 서민 민생대책위원회는 그의 발언이 국민을 선동하고, 검찰 수사에 해를 끼칠 수 있다며 유 이사장을 허위사실 유포와 위계에 의한 공무집행 방해 등 혐의로 1일 검찰에 고발했다.', '통상 허위사실 유포 및 명예훼손 등 사건은 형사 1부가 전담하지만, 처리해야 할 다른 사건이 많아 형사 3부에 이번 사건이 배당된 것으로 전해 졌다.', '유 이사장은 지난 8월 서울대에서 열린 ‘ 조국 사퇴 집회 ’를 두고 “ 뒤에서 자유 한국당 패거리의 손길이 어른 어른하는, 그런 것이라 본다” 고 했다가 고발당하기도 했다.', '유 이사장은 서울대 총학생회 주도로 열린 당시 시위를 향해 “ 물 반, 고기 반” 이라며 “ 